In [1]:
import requests
import pandas as pd
import time

In [159]:
# This api key will change. Update every 24 hours. 

api_key = "RGAPI-c1e1454a-d584-46fe-a337-2ecf95df27bd"

My: {
    "puuid": "Ucia41-iGooJ_05XmU32x3kHTSvTzU5kc9lgzXdPYx1mr1REVuK_kIynCbn2eYhZZkdWbUgZHCdsVA",
    "gameName": "Bigbobo",
    "tagLine": "8249"
}

Ben: {
    "puuid": "EYQmR6wL_lCyXJHLd7fGtOOM21ZpJt8l4Hguodx--vLyQJ3XZsLrgk2dG9gX7b3iKH30qzd1SYFPfA",
    "gameName": "Beniojinini",
    "tagLine": "6042"
}

### Getting PUUID for future use.

In [160]:
#URL from puuid
#Mine
URL = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/Ucia41-iGooJ_05XmU32x3kHTSvTzU5kc9lgzXdPYx1mr1REVuK_kIynCbn2eYhZZkdWbUgZHCdsVA"

#Ben
#URL = "https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/EYQmR6wL_lCyXJHLd7fGtOOM21ZpJt8l4Hguodx--vLyQJ3XZsLrgk2dG9gX7b3iKH30qzd1SYFPfA"

In [161]:
api_url = URL+"?api_key="+api_key
print(api_url)

https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/Ucia41-iGooJ_05XmU32x3kHTSvTzU5kc9lgzXdPYx1mr1REVuK_kIynCbn2eYhZZkdWbUgZHCdsVA?api_key=RGAPI-c1e1454a-d584-46fe-a337-2ecf95df27bd


In [162]:
respon= requests.get(api_url)


In [163]:
player_info = respon.json()
print(player_info)

{'id': 'HKgdDYFvZ3U_b9qZQReT0UT-5NAVE3TMd3fRYBZexcNz3x6vrMT4glrHqw', 'accountId': 'iEqFeBMdM5P8g9m1X3khZLHoUqsPzTQS5ohPmWBC2PwiiBLZ61DpcKrh', 'puuid': 'Ucia41-iGooJ_05XmU32x3kHTSvTzU5kc9lgzXdPYx1mr1REVuK_kIynCbn2eYhZZkdWbUgZHCdsVA', 'profileIconId': 1298, 'revisionDate': 1734928752000, 'summonerLevel': 46}


In [164]:
player_puuid = player_info["puuid"]
player_puuid

'Ucia41-iGooJ_05XmU32x3kHTSvTzU5kc9lgzXdPYx1mr1REVuK_kIynCbn2eYhZZkdWbUgZHCdsVA'

### Matches -- Using PUUID to get my match id

In [165]:
def match_url(start):
    

    #Mine
    api_match =( 
                "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/Ucia41-iGooJ_05XmU32x3kHTSvTzU5kc9lgzXdPYx1mr1REVuK_kIynCbn2eYhZZkdWbUgZHCdsVA/ids?start="
                +str(100*start)
                +"&count=100"
                )
    #Ben
    #api_match = "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/EYQmR6wL_lCyXJHLd7fGtOOM21ZpJt8l4Hguodx--vLyQJ3XZsLrgk2dG9gX7b3iKH30qzd1SYFPfA/ids?start=0&count=100"
    
    return api_match

#How many hundred games I want to get
N=3

def match_list(api_match):
    api_match = api_match+"&api_key="+api_key
    respon = requests.get(api_match)
    matches = respon.json()
    return (matches)
    
matches=[]    
for i in range(0, N):
    url = match_url(i)
    match_chunk = match_list(url)

    matches.extend(match_chunk)  # Extend the list with the results
    

In [166]:
def url_list(x):
    
    return  "https://americas.api.riotgames.com/lol/match/v5/matches/"+x+"?api_key="+api_key

# Use a lambda function with map to create the list of URLs
match_url_list = list(map(lambda r: url_list(r), matches))

In [167]:
match_url_list

['https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5189122840?api_key=RGAPI-c1e1454a-d584-46fe-a337-2ecf95df27bd',
 'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5188352721?api_key=RGAPI-c1e1454a-d584-46fe-a337-2ecf95df27bd',
 'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5188335385?api_key=RGAPI-c1e1454a-d584-46fe-a337-2ecf95df27bd',
 'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5186605508?api_key=RGAPI-c1e1454a-d584-46fe-a337-2ecf95df27bd',
 'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5184989201?api_key=RGAPI-c1e1454a-d584-46fe-a337-2ecf95df27bd',
 'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5184968868?api_key=RGAPI-c1e1454a-d584-46fe-a337-2ecf95df27bd',
 'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5184528391?api_key=RGAPI-c1e1454a-d584-46fe-a337-2ecf95df27bd',
 'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5184281364?api_key=RGAPI-c1e1454a-d584-46fe-a337-2ecf95d

### Look up stats using matchID -- Using one game only

In [168]:
respon = requests.get(match_url_list[0])

In [169]:
stats = respon.json()
print(stats)

{'metadata': {'dataVersion': '2', 'matchId': 'NA1_5189122840', 'participants': ['KT9y2BsvULhxvv4LwrE5lWVRA_zm7eOS7xR0NE0bTRwkstFB1peQMSp0HahcF5kWoMWfVAdmbdHtKA', 'G_05efYRexwzyoJW_JhviHk7rTkzOeUeJy3VOwmiF9vds7o71RmhJZWao3V-jTSAQcYSAFHtVyEVkA', 'kgTNc-jDdIpBKdV4HgOyg3WFjJrU303jOoxwOKbmkq9B5J7c1WG-tBPIK2rli-sgRwO69iNpqk61ow', 'XhZLOtWu0yiHjDeCYmkEw6KvF5CGUcj3J8QMX7obP61_gyFkc3jQx3Dj_zTNg6Jqjufm4tFogiHBAw', 'cIKYdI1pc8-QhLUqVqsFIow2m1Db-xdlrKGpwlY8wScdLMsNjkqVHYVVdyNQqnFMG96_k60wdMVCeQ', 'FJYOAGZ9pCoLdf1JzKd8M5YLf8P0eNLAe8nee_DKIxmlSU2EPbYYH_Y3ihBt2ii2NnbeURi5B6edUA', 'CqaomEdxLjVSX_xHBRggq-GRkFSozHqSulu6qwWsT-r8645FhGsuLPFd2P9SuauGs8SJP_nJ2RFHCw', 'MDcrXYmogx9VBydEOK12B-4irJjWLlXerZlhFspxev2U6YoRZEzmJoWC1YLQM2mQhXz1ZyZnigd2Gg', 'Ucia41-iGooJ_05XmU32x3kHTSvTzU5kc9lgzXdPYx1mr1REVuK_kIynCbn2eYhZZkdWbUgZHCdsVA', 'EYQmR6wL_lCyXJHLd7fGtOOM21ZpJt8l4Hguodx--vLyQJ3XZsLrgk2dG9gX7b3iKH30qzd1SYFPfA']}, 'info': {'endOfGameResult': 'GameComplete', 'gameCreation': 1734927186011, 'gameDuration': 1532, '

In [127]:
print(stats.keys())
player_puuid = player_info["puuid"]
print(player_puuid)


print(stats["metadata"])

dict_keys(['status'])


KeyError: 'puuid'

In [128]:
stats["info"]

KeyError: 'info'

In [65]:
pd.options.display.max_columns = None
stats["info"].keys()


dict_keys(['endOfGameResult', 'gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])

In [ ]:
### Community Dragon for item name.

In [2]:
item ="https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/items.json"
item_json = requests.get(item).json()
item_json


[{'id': 1001,
  'name': 'Boots',
  'description': '<mainText><stats><attention> 25</attention> Move Speed</stats><br><br></mainText>',
  'active': False,
  'inStore': True,
  'from': [],
  'to': [3117, 3158, 3047, 3111, 3006, 3005, 3009, 3010, 3020],
  'categories': ['Boots'],
  'maxStacks': 1,
  'requiredChampion': '',
  'requiredAlly': '',
  'requiredBuffCurrencyName': '',
  'requiredBuffCurrencyCost': 0,
  'specialRecipe': 0,
  'isEnchantment': False,
  'price': 300,
  'priceTotal': 300,
  'displayInItemSets': True,
  'iconPath': '/lol-game-data/assets/ASSETS/Items/Icons2D/1001_Class_T1_BootsofSpeed.png'},
 {'id': 1004,
  'name': 'Faerie Charm',
  'description': '<mainText><stats><attention> 50%</attention> Base Mana Regen</stats><br><br></mainText>',
  'active': False,
  'inStore': True,
  'from': [],
  'to': [3012, 3114, 4642],
  'categories': ['ManaRegen'],
  'maxStacks': 1,
  'requiredChampion': '',
  'requiredAlly': '',
  'requiredBuffCurrencyName': '',
  'requiredBuffCurrencyC

In [3]:
def json_extract(obj, key):
    arr = []

    def extract(obj, arr, key):
        if isinstance(obj, dict):
            #print(obj.items()) 
            for k, v in obj.items(): #obj.items() returns all key-value pairs of the dictionary obj. 
                if k == key:
                    arr.append(v)

        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key) #Returning array.
    return values

In [4]:
item_id = json_extract(item_json, "id")
item_name = json_extract(item_json, "name")

In [5]:
#map(function, *iterables):
item_dict = dict(map(lambda a, b: (int(a),b),item_id,item_name  ))


In [8]:
print(item_name)

['Boots', 'Faerie Charm', 'Rejuvenation Bead', "Giant's Belt", 'Cloak of Agility', 'Blasting Wand', 'Sapphire Crystal', 'Ruby Crystal', 'Cloth Armor', 'Chain Vest', 'Null-Magic Mantle', 'Emberknife', 'Long Sword', 'Pickaxe', 'B. F. Sword', 'Hailblade', 'Obsidian Edge', 'Dagger', 'Recurve Bow', 'Amplifying Tome', 'Vampiric Scepter', "Doran's Shield", "Doran's Blade", "Doran's Ring", 'Negatron Cloak', 'Needlessly Large Rod', 'Dark Seal', 'Cull', 'Scorchclaw Pup', 'Gustwalker Hatchling', 'Mosstomper Seedling', 'Eye of the Herald', 'Mosstomper Seedling', 'Gustwalker Hatchling', 'Scorchclaw Pup', 'Penetrating Bullets', 'Fortification', 'Reinforced Armor', "Warden's Eye", 'Vanguard', 'Reinforced Armor', 'Overcharged', 'Anti-tower Socks', 'Gusto', 'Phreakish Gusto', 'Super Mech Armor', 'Super Mech Power Field', 'Turret Plating', 'Structure Bounty', 'Structure Bounty', 'Structure Bounty', 'Structure Bounty', 'OvererchargedHA', 'Fortification', 'Tower Power-Up', 'Recall', 'Health Potion', '', '

In [10]:

gear_df = pd.DataFrame(item_name,columns=["item_name"])
gear_df

,item_name
0,Boots
1,Faerie Charm
2,Rejuvenation Bead
3,Giant's Belt
4,Cloak of Agility
...,...
601,Lightning Rod
602,Diamond-Tipped Spear
603,Twilight's Edge
604,Black Hole Gauntlet


In [13]:
gear_df.to_excel('gear_list_excel.xlsx', index=False)

### Look up stats using matchID -- Using all games

In [172]:

def games_combine(x, game_number):
    
    while True:
        
        response = requests.get(x)
        if response.status_code ==429:
            print(game_number)
            print("sleeping")
            time.sleep(30)
            continue
        if response.status_code == 200:
            break
    game_stats = response.json()
    DF_all = pd.DataFrame(game_stats["info"]["participants"])
    DF_all["game_number"] = game_number  # Add a new column to mark the game number
    
    if 'puuid' in DF_all.columns:
        Teamnumber = DF_all.loc[DF_all["puuid"] == player_puuid, "teamId"].iloc[0]
        DF_all["myteam_number"] = Teamnumber
    else:
        print("Warning: 'puuid' column not found.")
        DF_all["myteam_number"] = None  # Set a default value if 'puuid' is missing

    print(game_number)
    
    return DF_all

# Generate the combined DataFrame
DF_all = pd.concat(
    [games_combine(urls, game_number) for game_number, urls in enumerate(match_url_list, start=0)],
    ignore_index=True
)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
sleeping
96
sleeping
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
sleeping
196
sleeping
196
sleeping
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261

In [173]:
DF_all

,allInPings,assistMePings,assists,baronKills,basicPings,bountyLevel,challenges,champExperience,champLevel,championId,championName,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,eligibleForProgression,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,getBackPings,goldEarned,goldSpent,holdPings,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,missions,needVisionPings,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,onMyWayPings,participantId,pentaKills,perks,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,placement,playerAugment1,playerAugment2,playerAugment3,playerAugment4,playerAugment5,playerAugment6,playerSubteamId,profileIcon,pushPings,puuid,quadraKills,retreatPings,riotIdGameName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,subteamPlacement,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,game_number,myteam_number
0,2.0,5.0,21.0,0.0,0.0,13.0,"{'12AssistStreakCount': 0, 'HealFromMapSources...",28468.0,18.0,104.0,Graves,0.0,3.0,0.0,1480.0,1480.0,1480.0,31413.0,0.0,9.0,0.0,1.0,0.0,True,1.0,0.0,False,False,False,False,False,True,0.0,16559.0,16435.0,0.0,Invalid,1.0,1.0,2.0,3031.0,3033.0,3133.0,3072.0,3153.0,3006.0,0.0,22.0,4.0,13.0,TOP,575.0,5.0,2.0,236.0,3917.0,1681.0,10958.0,"{'playerScore0': 0, 'playerScore1': 0, 'player...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,"{'statPerks': {'defense': 5001, 'flex': 5010, ...",98640.0,18599.0,24028.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4249.0,0.0,KT9y2BsvULhxvv4LwrE5lWVRA_zm7eOS7xR0NE0bTRwkst...,0.0,0.0,a7wa7er,NA1,SUPPORT,0.0,71.0,42.0,40.0,13.0,0.0,9.0,7.0,6.0,4.0,4456yXpeha8liHne5_TS4aevT6pSv-5UXpJmHiT3cylsURNx,97.0,a7wa7er,False,100.0,,24.0,1532.0,0.0,105339.0,23063.0,0.0,37724.0,0.0,14863.0,959.0,96.0,297.0,274.0,5.0,0.0,2781.0,2781.0,2738.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0,200
1,0.0,0.0,13.0,0.0,0.0,26.0,"{'12AssistStreakCount': 0, 'HealFromMapSources...",24318.0,18.0,523.0,Aphelios,0.0,1.0,1.0,3315.0,3315.0,3315.0,42473.0,0.0,19.0,0.0,8.0,0.0,True,0.0,2.0,False,True,False,True,False,True,0.0,20109.0,19300.0,0.0,Invalid,0.0,1.0,2.0,6676.0,3031.0,6673.0,3072.0,6333.0,3047.0,2052.0,26.0,8.0,26.0,TOP,1061.0,3.0,3.0,169.0,2099.0,2042.0,16411.0,"{'playerScore0': 0, 'playerScore1': 0, 'player...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",116229.0,49318.0,29421.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,505.0,0.0,G_05efYRexwzyoJW_JhviHk7rTkzOeUeJy3VOwmiF9vds7...,0.0,1.0,MaikuKnight,HMNG,SUPPORT,0.0,73.0,114.0,0.0,10.0,0.0,9.0,6.0,7.0,4.0,-nMyhU53AeOxFMskXGQ5feRwCxCtjG7JAxDpiT83ccU6kr4,365.0,MaikuKnight,False,100.0,,21.0,1532.0,0.0,122710.0,53371.0,0.0,49552.0,0.0,11877.0,0.0,80.0,70.0,517.0,1.0,2.0,4380.0,2010.0,3719.0,1.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0,200
2,0.0,0.0,21.0,0.0,0.0,4.0,"{'12AssistStreakCount': 0, 'HealFromMapSources...",24814.0,18.0,202.0,Jhin,0.0,2.0,0

In [201]:

#Change item from float to actual name.
DF_all[["item0","item1","item2","item3","item4","item5","item6"]]=DF_all[["item0","item1","item2","item3","item4","item5","item6"]].astype(int)
DF_all["item1"].info()

DF_all = DF_all.replace(item_dict)

<class 'pandas.core.series.Series'>
RangeIndex: 2990 entries, 0 to 2989
Series name: item1
Non-Null Count  Dtype
--------------  -----
2990 non-null   int64
dtypes: int64(1)
memory usage: 23.5 KB


### Adding Teamates recognition.

In [202]:
def teamate_recog(myteam, team):
    try:
        if myteam == team:
            return 1
        else:
            return 0
    except Exception as e:
        return 0  # Default value for other errors

# Apply the function row-wise
DF_all["Teamates"] = DF_all.apply(lambda r: teamate_recog(r["myteam_number"], r["teamId"]), axis=1)

In [203]:
DF_all

,allInPings,assistMePings,assists,baronKills,basicPings,bountyLevel,challenges,champExperience,champLevel,championId,championName,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,eligibleForProgression,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,getBackPings,goldEarned,goldSpent,holdPings,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,missions,needVisionPings,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,onMyWayPings,participantId,pentaKills,perks,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,placement,playerAugment1,playerAugment2,playerAugment3,playerAugment4,playerAugment5,playerAugment6,playerSubteamId,profileIcon,pushPings,puuid,quadraKills,retreatPings,riotIdGameName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,subteamPlacement,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,game_number,myteam_number,Teamates
0,2.0,5.0,21.0,0.0,0.0,13.0,"{'12AssistStreakCount': 0, 'HealFromMapSources...",28468.0,18.0,104.0,Graves,0.0,3.0,0.0,1480.0,1480.0,1480.0,31413.0,0.0,9.0,0.0,1.0,0.0,True,1.0,0.0,False,False,False,False,False,True,0.0,16559.0,16435.0,0.0,Invalid,1.0,1.0,2.0,Infinity Edge,Mortal Reminder,Caulfield's Warhammer,Bloodthirster,Blade of The Ruined King,Berserker's Greaves,0,22.0,4.0,13.0,TOP,575.0,5.0,2.0,236.0,3917.0,1681.0,10958.0,"{'playerScore0': 0, 'playerScore1': 0, 'player...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,"{'statPerks': {'defense': 5001, 'flex': 5010, ...",98640.0,18599.0,24028.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4249.0,0.0,KT9y2BsvULhxvv4LwrE5lWVRA_zm7eOS7xR0NE0bTRwkst...,0.0,0.0,a7wa7er,NA1,SUPPORT,0.0,71.0,42.0,40.0,13.0,0.0,9.0,7.0,6.0,4.0,4456yXpeha8liHne5_TS4aevT6pSv-5UXpJmHiT3cylsURNx,97.0,a7wa7er,False,100.0,,24.0,1532.0,0.0,105339.0,23063.0,0.0,37724.0,0.0,14863.0,959.0,96.0,297.0,274.0,5.0,0.0,2781.0,2781.0,2738.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0,200,0
1,0.0,0.0,13.0,0.0,0.0,26.0,"{'12AssistStreakCount': 0, 'HealFromMapSources...",24318.0,18.0,523.0,Aphelios,0.0,1.0,1.0,3315.0,3315.0,3315.0,42473.0,0.0,19.0,0.0,8.0,0.0,True,0.0,2.0,False,True,False,True,False,True,0.0,20109.0,19300.0,0.0,Invalid,0.0,1.0,2.0,The Collector,Infinity Edge,Immortal Shieldbow,Bloodthirster,Death's Dance,Plated Steelcaps,Poro-Snax,26.0,8.0,26.0,TOP,1061.0,3.0,3.0,169.0,2099.0,2042.0,16411.0,"{'playerScore0': 0, 'playerScore1': 0, 'player...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",116229.0,49318.0,29421.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,505.0,0.0,G_05efYRexwzyoJW_JhviHk7rTkzOeUeJy3VOwmiF9vds7...,0.0,1.0,MaikuKnight,HMNG,SUPPORT,0.0,73.0,114.0,0.0,10.0,0.0,9.0,6.0,7.0,4.0,-nMyhU53AeOxFMskXGQ5feRwCxCtjG7JAxDpiT83ccU6kr4,365.0,MaikuKnight,False,100.0,,21.0,1532.0,0.0,122710.0,53371.0,0.0,49552.0,0.0,11877.0,0.0,80.0,70.0,517.0,1.0,2.0,4380.0,Total Biscuit of Everlasting Will,3719.0,1.

In [204]:
DF_all.to_excel('my'+str(100*N)+'games.xlsx', index=False)